# Pythonic Filters
This notebook demonstrates how to use pythonic filters to query your Chroma collection.
> Note: Pythonic filters currently apply only to `where` (metadata) filters.

In [1]:
import chromadb
client = chromadb.Client()
# Create a new chroma collection
collection_name = "filter_example_collection"
collection = client.get_or_create_collection(name=collection_name)
# Add some data to the collection
collection.add(
    embeddings=[
        [1.1, 2.3, 3.2],
        [4.5, 6.9, 4.4],
        [1.1, 2.3, 3.2],
        [4.5, 6.9, 4.4],
        [1.1, 2.3, 3.2],
        [4.5, 6.9, 4.4],
        [1.1, 2.3, 3.2],
        [4.5, 6.9, 4.4],
    ],
    metadatas=[
        {"status": "read","rating": 0.6},
        {"status": "unread","rating": 0.1},
        {"status": "read","rating": 0.9},
        {"status": "unread","rating": 0.34},
        {"status": "read","rating": 0.28},
        {"status": "unread","rating": 0.499},
        {"status": "read","rating": 0.91},
        {"status": "unread","rating": 0.41},
    ],
    documents=["A document that discusses domestic policy", "A document that discusses international affairs", "A document that discusses kittens", "A document that discusses dogs", "A document that discusses chocolate", "A document that is sixth that discusses government", "A document that discusses international affairs", "A document that discusses global affairs"],
    ids=["id1", "id2", "id3", "id4", "id5", "id6", "id7", "id8"],
)
from chromadb import WhereFilter
# Get documents that are read and about affairs
simple_query=collection.get(
    where=WhereFilter("status"== "read"), 
    where_document={"$contains": "affairs"})

multi_line=collection.get(
    where=WhereFilter("status"== "read" 
                       or "status"== "unread"),
    where_document={"$contains": "affairs"})

print(simple_query)
print(multi_line)

{'ids': ['id7'], 'embeddings': None, 'metadatas': [{'rating': 0.91, 'status': 'read'}], 'documents': ['A document that discusses international affairs']}
{'ids': ['id2', 'id7', 'id8'], 'embeddings': None, 'metadatas': [{'rating': 0.1, 'status': 'unread'}, {'rating': 0.91, 'status': 'read'}, {'rating': 0.41, 'status': 'unread'}], 'documents': ['A document that discusses international affairs', 'A document that discusses international affairs', 'A document that discusses global affairs']}


# Filtering With Numeric Values

While string values with equal/not equal operators are easily supported by the syntax above, Python will throw an exception for comparing string to int or float (e.g. `"a" > 1`). To compare numeric values we introduce attribute (`Attr`) objects. Which allow you to perform numeric comparisons on metadata values using the following syntax:

> Note: Both `attr.rating` and `attr["rating"]` are valid.

In [3]:
from chromadb.utils.query_helper import attr

q = WhereFilter(attr.rating > 0.1 and attr["rating"] <= 0.9)
numeric_filter=collection.get(where=WhereFilter(attr.rating > 0.5 and attr["rating"] <= 0.9))
print(q)
print(numeric_filter)

{'$and': [{'rating': {'$gt': 0.1}}, {'rating': {'$lte': 0.9}}]}
{'ids': ['id1', 'id3'], 'embeddings': None, 'metadatas': [{'rating': 0.6, 'status': 'read'}, {'rating': 0.9, 'status': 'read'}], 'documents': ['A document that discusses domestic policy', 'A document that discusses kittens']}


# Where Filtering With Logical Operators
This section demonstrates how one can use the logical operators in `where` filtering.

Chroma currently supports: `$and` and `$or` operators.

> Note: Logical operators can be nested

In [9]:
# Or Logical Operator Filtering
# import chromadb
client = chromadb.Client()
collection = client.get_or_create_collection("test-where-list")
collection.add(documents=["Article by john", "Article by Jack", "Article by Jill"],
               metadatas=[{"author": "john"}, {"author": "jack"}, {"author": "jill"}], ids=["1", "2", "3"])

q1=collection.get(where={"$or": [{"author": "john"}, {"author": "jack"}]})
q2=collection.get(where=WhereFilter("author"=="john" or "author"=="jack"))
print(q1)
print(q2)

{'ids': ['1', '2'], 'embeddings': None, 'metadatas': [{'author': 'john'}, {'author': 'jack'}], 'documents': ['Article by john', 'Article by Jack']}
{'ids': ['1', '2'], 'embeddings': None, 'metadatas': [{'author': 'john'}, {'author': 'jack'}], 'documents': ['Article by john', 'Article by Jack']}


In [10]:
# And Logical Operator Filtering
collection = client.get_or_create_collection("test-where-list")
collection.upsert(documents=["Article by john", "Article by Jack", "Article by Jill"],
               metadatas=[{"author": "john","category":"chroma"}, {"author": "jack","category":"ml"}, {"author": "jill","category":"lifestyle"}], ids=["1", "2", "3"])
q1=collection.get(where={"$and": [{"category": "chroma"}, {"author": "john"}]})
q2=collection.get(where=WhereFilter("category"=="chroma" and "author"=="john"))
print(q1)
print(q2)

{'ids': ['1'], 'embeddings': None, 'metadatas': [{'author': 'john', 'category': 'chroma'}], 'documents': ['Article by john']}
{'ids': ['1'], 'embeddings': None, 'metadatas': [{'author': 'john', 'category': 'chroma'}], 'documents': ['Article by john']}


In [11]:
# And logical that doesn't match anything
q1=collection.get(where={"$and": [{"category": "chroma"}, {"author": "jill"}]})
q2=collection.get(where=WhereFilter("category"=="chroma" and "author"=="jill"))
print(q1)
print(q2)

{'ids': [], 'embeddings': None, 'metadatas': [], 'documents': []}
{'ids': [], 'embeddings': None, 'metadatas': [], 'documents': []}


In [12]:
# Combined `And` and `Or` Logical Operator Filtering
q1=collection.get(where={"$and": [{"category": "chroma"}, {"$or": [{"author": "john"}, {"author": "jack"}]}]})
q2=collection.get(where=WhereFilter("category"=="chroma" and ("author"=="john" or "author"=="jack")))
print(q1)
print(q2)

{'ids': ['1'], 'embeddings': None, 'metadatas': [{'author': 'john', 'category': 'chroma'}], 'documents': ['Article by john']}
{'ids': ['1'], 'embeddings': None, 'metadatas': [{'author': 'john', 'category': 'chroma'}], 'documents': ['Article by john']}
